In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from shapely.geometry import Point
df = gpd.read_file('/data/acker/ALA/NA/NAv5_GWRPM25_mean2016-2018.shp')
#df = gpd.read_file('/data/acker/ALA/NA/NAv5_GWRPM25_mean2016-2018_99th.shp')
df

,lat,lon,GWRPM25,geometry
0,14.005,-91.455,29.000000,POINT (-91.455 14.005)
1,14.005,-91.445,28.966667,POINT (-91.445 14.005)
2,14.005,-91.435,28.966667,POINT (-91.435 14.005)
3,14.005,-91.425,28.766668,POINT (-91.425 14.005)
4,14.005,-91.415,28.766668,POINT (-91.415 14.005)
...,...,...,...,...
24963547,67.995,-64.735,0.300000,POINT (-64.735 67.995)
24963548,67.995,-64.725,0.300000,POINT (-64.725 67.995)
24963549,67.995,-64.715,0.300000,POINT (-64.715 67.995)
24963550,67.995,-64.705,0.266667,POINT (-64.705 67.995)


In [2]:
df['GWRPM25'].max()

63.133331298828125

In [3]:
#upload shapefile of U.S. counties
counties = gpd.read_file('/data/acker/shapefiles/cb_2020_us_county_500k.shp')

In [4]:
# List of state abbreviations for CONUS, Alaska (AK), and Hawaii (HI)
states_to_include = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'DC',
    'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
    'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
    'WI', 'WY'
]

# Filter counties to only include rows where STUSPS is in the specified list
counties_conus = counties[counties['STUSPS'].isin(states_to_include)]

# Display the filtered counties
counties_conus

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,13,031,00350302,0500000US13031,13031,Bulloch,Bulloch County,GA,Georgia,06,1750735216,32827779,"POLYGON ((-82.02684 32.55516, -82.02527 32.558..."
1,13,121,01694833,0500000US13121,13121,Fulton,Fulton County,GA,Georgia,06,1364133162,19666288,"POLYGON ((-84.84931 33.51318, -84.84429 33.514..."
2,13,179,00357095,0500000US13179,13179,Liberty,Liberty County,GA,Georgia,06,1337654102,222884791,"POLYGON ((-81.8244 32.01488, -81.81338 32.0162..."
3,13,189,00348794,0500000US13189,13189,McDuffie,McDuffie County,GA,Georgia,06,666590014,23114032,"POLYGON ((-82.64852 33.60838, -82.64409 33.606..."
4,13,213,01688005,0500000US13213,13213,Murray,Murray County,GA,Georgia,06,892235145,5719644,"POLYGON ((-84.94434 34.68004, -84.9431 34.6804..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,41,049,01135860,0500000US41049,41049,Morrow,Morrow County,OR,Oregon,06,5259046087,44372730,"POLYGON ((-120.00631 45.27648, -120.00556 45.3..."
3229,21,073,00516883,0500000US21073,21073,Franklin,Franklin County,KY,Kentucky,06,538312209,10786432,"POLYGON ((-85.02426 38.13059, -85.02299 38.131..."
3230,17,201,01785216,0500000US17201,17201,Winnebago,Winnebago County,IL,Illinois,06,1328792556,15873993,"POLYGON ((-89.40142 42.50043, -89.37175 42.500..."
3231,17,123,00424260,0500000US17123,17123,Marshall,Marshall County,IL,Illinois,06,1001843226,30364348,"POLYGON ((-89.63869 40.98691, -89.63868 40.988..."


In [5]:
counties_conus = counties_conus.drop(["COUNTYNS", 'NAMELSAD', 'LSAD', 'ALAND', 'AWATER', 'AFFGEOID'], axis=1)

In [6]:
counties_conus = counties_conus.to_crs(df.crs)

In [7]:
import geopandas as gpd
import pandas as pd

# Parameters
chunk_size = 10000  # Adjust this based on your memory capacity

# Initialize an empty list to store results
results = []

# Iterate over chunks of the data
for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i+chunk_size]
    chunk_result = gpd.sjoin(chunk, counties_conus[['GEOID', 'geometry','STATEFP','COUNTYFP']], how="left", predicate="within")
    chunk_result.dropna(subset=['GEOID'], inplace=True)  # Drop rows where GEOID is NaN in this chunk
    results.append(chunk_result)

# Concatenate all chunk results
df_counties = pd.concat(results, ignore_index=True)


In [8]:
df_counties

,lat,lon,GWRPM25,geometry,index_right,GEOID,STATEFP,COUNTYFP
0,18.915,-155.675,5.933333,POINT (-155.675 18.915),269.0,15001,15,001
1,18.925,-155.675,5.733334,POINT (-155.675 18.925),269.0,15001,15,001
2,18.925,-155.665,5.633333,POINT (-155.665 18.925),269.0,15001,15,001
3,18.925,-155.655,5.533333,POINT (-155.655 18.925),269.0,15001,15,001
4,18.935,-155.675,5.666667,POINT (-155.675 18.935),269.0,15001,15,001
...,...,...,...,...,...,...,...,...
10390484,67.995,-141.045,2.900000,POINT (-141.045 67.995),2780.0,02290,02,290
10390485,67.995,-141.035,2.866667,POINT (-141.035 67.995),2780.0,02290,02,290
10390486,67.995,-141.025,2.833333,POINT (-141.025 67.995),2780.0,02290,02,290
10390487,67.995,-141.015,2.800000,POINT (-141.015 67.995),2780.0,02290,02,290


In [9]:
# Group by GEOID and calculate the 90th percentile of GWRPM25 for each county
#county_90th_percentile = df_counties.groupby('GEOID')['GWRPM25'].quantile(0.9).reset_index()
county_90th_percentile = df_counties.groupby('GEOID')['GWRPM25'].max().reset_index()
#county_90th_percentile = df_counties.groupby('GEOID')['GWRPM25'].mean().reset_index()


# Rename the column for clarity
county_90th_percentile = county_90th_percentile.rename(columns={'GWRPM25': 'PM25_90th'})

# Display the result
print(county_90th_percentile)


      GEOID  PM25_90th
0     01001   8.566667
1     01003   9.133333
2     01005   7.866667
3     01007   8.099999
4     01009   8.233334
...     ...        ...
3137  56037   6.933333
3138  56039   6.466667
3139  56041   7.566667
3140  56043   6.466667
3141  56045  10.266666

[3142 rows x 2 columns]


In [10]:
# Merge the 90th percentile data with the counties GeoDataFrame to include geometry
county_90th_percentile_with_geometry = county_90th_percentile.merge(counties[['GEOID', 'geometry','STATEFP','COUNTYFP']], on='GEOID', how='left')

# Convert to a GeoDataFrame if necessary
county_90th_percentile_with_geometry = gpd.GeoDataFrame(county_90th_percentile_with_geometry, geometry='geometry')

# Display the result
print(county_90th_percentile_with_geometry)


      GEOID  PM25_90th                                           geometry  \
0     01001   8.566667  POLYGON ((-86.9212 32.65754, -86.92035 32.6585...   
1     01003   9.133333  POLYGON ((-88.02858 30.22676, -88.02399 30.230...   
2     01005   7.866667  POLYGON ((-85.74803 31.61918, -85.74544 31.618...   
3     01007   8.099999  POLYGON ((-87.42194 33.00338, -87.33177 33.005...   
4     01009   8.233334  POLYGON ((-86.96336 33.85822, -86.95967 33.857...   
...     ...        ...                                                ...   
3137  56037   6.933333  POLYGON ((-110.05438 42.01103, -110.05436 42.0...   
3138  56039   6.466667  POLYGON ((-111.05361 44.66627, -110.75076 44.6...   
3139  56041   7.566667  POLYGON ((-111.04663 41.14012, -111.04662 41.1...   
3140  56043   6.466667  POLYGON ((-108.55063 44.15179, -108.55056 44.1...   
3141  56045  10.266666  POLYGON ((-105.08123 43.65128, -105.0808 43.71...   

     STATEFP COUNTYFP  
0         01      001  
1         01      003  
2  

In [11]:
# epa CDVs for pearson
gdf_epa = pd.read_csv('/data/acker/EPA_DV/2016-2018.csv')

In [12]:
gdf_epa

,State FIPS,County FIPS,State,County,2016-2018
0,1,3,AL,Baldwin,7.3
1,1,27,AL,Clay,7.4
2,1,33,AL,Colbert,7.5
3,1,49,AL,DeKalb,7.6
4,1,55,AL,Etowah,8.3
...,...,...,...,...,...
538,56,35,WY,Sublette,5.3
539,56,37,WY,Sweetwater,5.3
540,56,39,WY,Teton,4.8
541,72,61,PR,Guaynabo,NaN


In [13]:
gdf_epa.rename(columns={'2016-2018': 'Design Value'}, inplace=True)

In [14]:

# Function to add leading zero to 11-digit codes
def add_leading_zero(code):
    code_str = str(code).strip()
    if len(code_str) == 1:
        ret = '0' + code_str
        return ret
    return code_str

In [15]:
# Iterate through each block group ID in ejscreen and update if necessary
for i in range(len(gdf_epa)):
    gdf_epa.at[i, 'State FIPS'] = add_leading_zero(gdf_epa.at[i, 'State FIPS'])

# Iterate through each block group ID in ejscreen and update if necessary
for i in range(len(gdf_epa)):
    gdf_epa.at[i, 'County FIPS'] = add_leading_zero(gdf_epa.at[i, 'County FIPS'])


/tmp/ipykernel_37395/958764295.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf_epa.at[i, 'State FIPS'] = add_leading_zero(gdf_epa.at[i, 'State FIPS'])
/tmp/ipykernel_37395/958764295.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '03' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf_epa.at[i, 'County FIPS'] = add_leading_zero(gdf_epa.at[i, 'County FIPS'])


In [16]:

# Function to add leading zero to 11-digit codes
def add_leading_zero(code):
    code_str = str(code).strip()
    if len(code_str) == 2:
        ret = '0' + code_str
        return ret
    return code_str


for i in range(len(gdf_epa)):
    gdf_epa.at[i, 'County FIPS'] = add_leading_zero(gdf_epa.at[i, 'County FIPS'])


In [17]:
county_90th_percentile_with_geometry.columns

Index(['GEOID', 'PM25_90th', 'geometry', 'STATEFP', 'COUNTYFP'], dtype='object')

In [18]:
import numpy as np

# Set values that are 0 in 'Design Value' to NaN in gdf_epa
gdf_epa.loc[gdf_epa['Design Value'] == 0, 'Design Value'] = np.nan

# Drop rows where 'Design Value' is NaN, so only counties with valid EPA data remain
gdf_epa.dropna(subset=['Design Value'], inplace=True)

# Step 1: Create a unique identifier combining 'STUSPS' and 'NAME' for both datasets
gdf_epa['ID'] = gdf_epa['State FIPS'] + "_" + gdf_epa['County FIPS']
county_90th_percentile_with_geometry['ID'] = county_90th_percentile_with_geometry['STATEFP'] + "_" + county_90th_percentile_with_geometry['COUNTYFP']

# Step 2: Identify IDs with valid values in gdf_epa
epa_ids = gdf_epa['ID'].unique()

# Step 3: Identify unmonitored counties
unmonitored_counties = county_90th_percentile_with_geometry[
    ~county_90th_percentile_with_geometry['ID'].isin(epa_ids)
]

# Step 4: Update PM25_90th in county_90th_percentile_with_geometry to NaN for IDs not in epa_ids
county_90th_percentile_with_geometry.loc[
    ~county_90th_percentile_with_geometry['ID'].isin(epa_ids), 'PM25_90th'
] = np.nan


In [19]:
len(epa_ids)

537

In [20]:
unmonitored_counties

,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID
0,01001,8.566667,"POLYGON ((-86.9212 32.65754, -86.92035 32.6585...",01,001,01_001
2,01005,7.866667,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",01,005,01_005
3,01007,8.099999,"POLYGON ((-87.42194 33.00338, -87.33177 33.005...",01,007,01_007
4,01009,8.233334,"POLYGON ((-86.96336 33.85822, -86.95967 33.857...",01,009,01_009
5,01011,8.133333,"POLYGON ((-85.99926 32.25018, -85.98655 32.250...",01,011,01_011
...,...,...,...,...,...,...
3132,56027,9.633333,"POLYGON ((-104.89994 43.49964, -104.78095 43.5...",56,027,56_027
3134,56031,6.733334,"POLYGON ((-105.28651 42.60583, -105.25054 42.6...",56,031,56_031
3139,56041,7.566667,"POLYGON ((-111.04663 41.14012, -111.04662 41.1...",56,041,56_041
3140,56043,6.466667,"POLYGON ((-108.55063 44.15179, -108.55056 44.1...",56,043,56_043


In [21]:
county_90th_percentile_with_geometry.dropna(inplace=True)

In [22]:
# Rank counties from highest to lowest based on PM25_90th values
county_ranked_epa = gdf_epa.sort_values(by='Design Value', ascending=False).reset_index(drop=True)
county_ranked_epa.reset_index(inplace=True)
county_ranked_epa['Design Value']

0      17.8
1      16.8
2      16.1
3      15.0
4      14.7
       ... 
532     3.6
533     3.4
534     3.4
535     2.9
536     2.9
Name: Design Value, Length: 537, dtype: float64

In [23]:
# Rank counties from highest to lowest based on PM25_90th values
county_ranked = county_90th_percentile_with_geometry.sort_values(by='PM25_90th', ascending=False).reset_index(drop=True)
county_ranked.reset_index(inplace=True)
county_ranked

,index,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID
0,0,49045,62.333332,"POLYGON ((-114.04713 39.90693, -114.04666 39.9...",49,045,49_045
1,1,49011,56.100002,"POLYGON ((-112.45162 41.08733, -112.19934 41.1...",49,011,49_011
2,2,35013,45.899998,"POLYGON ((-107.29963 32.60537, -107.2994 32.76...",35,013,35_013
3,3,49003,37.066666,"POLYGON ((-114.04255 41.21092, -114.0414 41.21...",49,003,49_003
4,4,49035,35.266666,"POLYGON ((-112.26022 40.76909, -112.19598 40.8...",49,035,49_035
...,...,...,...,...,...,...,...
532,532,27075,4.433333,"MULTIPOLYGON (((-91.55375 47.08965, -91.55327 ...",27,075,27_075
533,533,38007,4.366667,"POLYGON ((-103.66748 47.26888, -103.66668 47.3...",38,007,38_007
534,534,27031,4.166667,"MULTIPOLYGON (((-89.59206 47.96668, -89.59147 ...",27,031,27_031
535,535,15003,3.366667,"MULTIPOLYGON (((-157.65982 21.32857, -157.6588...",15,003,15_003


In [24]:
np.nanmax(gdf_epa['Design Value'])

17.8

In [25]:
np.nanmax(county_90th_percentile_with_geometry['PM25_90th'])

62.33333206176758

In [26]:
import numpy as np
from scipy.stats import rankdata

# Merge the two DataFrames on 'GEOID' to get matched pairs for CDV and CDVE
# CDV (CDV) = Design Value from county_ranked_epa
# CDVE (calculated satellite value) = PM25_90th from county_ranked
merged_df = county_ranked.merge(county_ranked_epa[['ID', 'Design Value']], on='ID', how='inner')

# Drop rows with NaN values in either column, only keep counties with both values
merged_df.dropna(subset=['PM25_90th', 'Design Value'], inplace=True)

# Extract CDV and CDVE values for correlation calculations
cdv = merged_df['Design Value'].values  # EPA Design Values (CDV)
cdve = merged_df['PM25_90th'].values  # 90th percentile PM2.5 values (CDVE)

# Calculate Pearson correlation coefficient
cdv_mean = np.mean(cdv)
cdve_mean = np.mean(cdve)
numerator = np.sum((cdv - cdv_mean) * (cdve - cdve_mean))
denominator = np.sqrt(np.sum((cdv - cdv_mean)**2) * np.sum((cdve - cdve_mean)**2))
pearson_correlation = numerator / denominator

# Calculate Spearman rank correlation coefficient
cdv_ranks = rankdata(cdv, method='min')
cdve_ranks = rankdata(cdve, method='min')
d_i = cdv_ranks - cdve_ranks
n = len(cdv)
spearman_numerator = 6 * np.sum(d_i**2)
spearman_denominator = n * (n**2 - 1)
spearman_correlation = 1 - (spearman_numerator / spearman_denominator)

# Print the results
print("Pearson Correlation Coefficient (r):", pearson_correlation)
print("Spearman Rank Correlation Coefficient (r_s):", spearman_correlation)


Pearson Correlation Coefficient (r): 0.3672992562666515
Spearman Rank Correlation Coefficient (r_s): 0.7286733943623247


In [27]:
from scipy.stats import pearsonr, spearmanr

pearson_r, _ = pearsonr(cdv, cdve)
spearman_r, _ = spearmanr(cdv, cdve)

print("Pearson Correlation (scipy):", pearson_r)
print("Spearman Correlation (scipy):", spearman_r)

Pearson Correlation (scipy): 0.3672992562666515
Spearman Correlation (scipy): 0.7289968750512571


In [28]:
merged_df.head(10)

,index,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID,Design Value
0,0,49045,62.333332,"POLYGON ((-114.04713 39.90693, -114.04666 39.9...",49,045,49_045,7.0
1,1,49011,56.100002,"POLYGON ((-112.45162 41.08733, -112.19934 41.1...",49,011,49_011,8.3
2,2,35013,45.899998,"POLYGON ((-107.29963 32.60537, -107.2994 32.76...",35,013,35_013,8.3
3,3,49003,37.066666,"POLYGON ((-114.04255 41.21092, -114.0414 41.21...",49,003,49_003,7.7
4,4,49035,35.266666,"POLYGON ((-112.26022 40.76909, -112.19598 40.8...",49,035,49_035,8.8
5,5,06093,28.166666,"POLYGON ((-123.71845 41.59796, -123.7188 41.59...",06,093,06_093,10.1
6,6,49057,23.300001,"POLYGON ((-112.49352 41.07689, -112.36894 41.2...",49,057,49_057,8.3
7,7,06037,19.766666,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",06,037,06_037,12.7
8,8,06023,18.800001,"POLYGON ((-124.4086 40.4432, -124.39664 40.462...",06,023,06_023,7.4
9,9,41033,18.166666,"POLYGON ((-124.04198 42.38766, -124.03884 42.3...",41,033,41_033,10.3


In [61]:
merged_df[merged_df['ID'] == '55_025']

,index,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID,Design Value
316,316,55025,8.166667,"POLYGON ((-89.83856 42.94994, -89.83854 42.955...",55,025,55_025,8.1


In [62]:
import pandas as pd

# Load your data into a DataFrame
# Assuming the DataFrame is already loaded and named `df`
gdf_epa.sort_values(by='ID', ascending=True, inplace=True)
county_90th_percentile_with_geometry.sort_values(by='ID', ascending=True, inplace=True)

# Drop rows with NaN in either the CDV or CDVE columns
filtered_df = gdf_epa.copy()
# Calculate the difference (bias for each row)
filtered_df['Difference'] = county_90th_percentile_with_geometry['PM25_90th'].values - gdf_epa['Design Value'].values

# Calculate the median bias instead of the mean
bias_median = filtered_df['Difference'].median()
print(filtered_df['Difference'].describe())
# Check for NaN values in the Difference column
print("NaN values in Difference:", filtered_df['Difference'].isna().sum())


# Calculate the bias range (min and max differences)
bias_min = filtered_df['Difference'].min()
bias_max = filtered_df['Difference'].max()

print(f"Bias (median): {bias_median:.2f} µg/m³")
print(f"Bias range: {bias_min:.2f} µg/m³ to {bias_max:.2f} µg/m³")


count    5.370000e+02
mean     1.251273e+00
std      4.268357e+00
min     -4.166666e+00
25%      1.907349e-07
50%      6.333330e-01
75%      1.333333e+00
max      5.533333e+01
Name: Difference, dtype: float64
NaN values in Difference: 0
Bias (median): 0.63 µg/m³
Bias range: -4.17 µg/m³ to 55.33 µg/m³


In [39]:
filtered_df['Difference'].isna().sum()

0